In [1]:
import numpy as np

# using truncated multivariate normal
# https://stackoverflow.com/questions/20115917/truncated-multivariate-normal-in-scipy
import emcee

In [2]:
from numpy.linalg import inv, cholesky
from scipy.linalg import qr

def lnprob_trunc_norm(x, mean, bounds, C):
    if np.any(x < bounds[:,0]) or np.any(x > bounds[:,1]):
        return -np.inf
    else:
        return -0.5*(x-mean).dot(inv(C)).dot(x-mean)

In [3]:
def multivariate_trunc_normal(mean, cov, bounds, size=1):
    ndim = mean.shape[0]
    nwalkers = max(10*ndim, size)
    nsteps = 1000 + size
    
    S = emcee.EnsembleSampler(nwalkers, ndim, lnprob_trunc_norm, args = (mean, bounds, cov))
    pos = emcee.utils.sample_ball(mean, np.sqrt(np.diag(C)), size=nwalkers)
    pos, prob, state = S.run_mcmc(pos, nsteps)
    
    return pos[-size:].flatten()

In [4]:
ndim = 10
mean = (np.random.rand(ndim)-1)*3
bounds=np.ones((ndim, 2))
bounds[:, 0] = -3
bounds[:, 1] = 3
C = np.eye(ndim)
theta = multivariate_trunc_normal(mean, C, bounds)
theta

c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
c:\users\chapm\anaconda3\lib\site-packages\emcee\ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))


array([-0.47463216, -2.07098625, -2.24876379, -0.21556144, -0.36336838,
       -1.03788551, -0.7309861 , -1.12364921, -0.93816597, -1.30960721])

Now we will have a look at RJMCMC techniques here.

In this notebook, we will consider "AutoRJ" where the parameters can be assumed to follow some kind of random walk and the first and second moments are known. 

In [5]:
mu1 = (np.random.rand(5)-1)*3
C1 = C[:5, :5]

B = cholesky(C)
B1 = cholesky(C1)

# now we sample u1 as standard normal.
u1 = np.random.normal(size=5)

In [6]:
R, _ = qr(B)
rank_part = R.dot(inv(B)).dot((theta - mean).flatten())
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1

array([-0.12329158, -1.03434802, -2.1483695 , -2.83467707, -0.85283769])

In [7]:
# if we assume that we will grow to 15 dimensions instead
mu1 = (np.random.rand(15)-1)*3
C1 = np.eye(15)
B1 = cholesky(C1)

# now we sample u1 as standard normal
u1 = np.random.normal(size=15)

In [8]:
R, _ = qr(B1)

inner_part = inv(B).dot((theta - mean).flatten())
rank_part = R.dot(np.hstack([inner_part, mu1[-5:]]))

In [9]:
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1

array([ -3.09866123e-01,  -2.47523416e+00,  -2.50988153e+00,
        -2.62094056e+00,  -1.05753724e+00,   3.47758642e-03,
        -2.37487040e+00,  -2.19841656e+00,  -1.06932378e+00,
         4.54446945e-01,  -5.80812946e+00,  -1.27213118e+00,
        -5.49820464e-01,  -9.68925120e-01,  -5.60013656e+00])